In [0]:
%run ./01-config

In [0]:
class Upserter:
    def __init__(self, merge_query, temp_view_name):
        self.merge_query = merge_query
        self.temp_view_name = temp_view_name 
        
    def upsert(self, df_micro_batch, batch_id):
        df_micro_batch.createOrReplaceTempView(self.temp_view_name)
        df_micro_batch._jdf.sparkSession().sql(self.merge_query)

# COMMAND ----------

class Gold():
    def __init__(self, env):
        self.Conf = Config() 
        self.checkpoint_base = self.Conf.base_dir_checkpoint + "/checkpoints"
        self.catalog = env
        self.db_sv_name = "ecommerce_db_sv"
        self.db_gd_name = "ecommerce_db_gd"
        self.maxFilesPerTrigger = self.Conf.maxFilesPerTrigger
        spark.sql(f"USE {self.catalog}.{self.db_gd_name}")

    def cleanup(self): 
        print(f"Đang xóa {self.checkpoint_base}...", end='')
        dbutils.fs.rm(f"{self.checkpoint_base}/dim_customer_gd", True)
        dbutils.fs.rm(f"{self.checkpoint_base}/dim_product_gd", True)
        dbutils.fs.rm(f"{self.checkpoint_base}/dim_seller_gd", True)
        dbutils.fs.rm(f"{self.checkpoint_base}/dim_time_gd", True)
        dbutils.fs.rm(f"{self.checkpoint_base}/fact_order_gd", True)
        dbutils.fs.rm(f"{self.checkpoint_base}/fact_sale_gd", True)

        print("Hoàn thành!")  

    def upsert_dim_customer_gd(self, once=True, processing_time="15 seconds"):
        from pyspark.sql import functions as F

        query = f"""
            MERGE INTO {self.catalog}.{self.db_gd_name}.dim_customer_gd target
            USING customer_delta source
            ON target.CustomerID = source.customer_id

            WHEN MATCHED THEN UPDATE SET
                target.CustomerCity = source.customer_city,
                target.CustomerState = source.customer_state

            WHEN NOT MATCHED THEN INSERT (CustomerID, CustomerCity, CustomerState)
            VALUES (source.customer_id, source.customer_city, source.customer_state)
        """

        data_upserter = Upserter(query, "customer_delta")

        df_delta = (
            spark.readStream
                .table(f"{self.catalog}.{self.db_sv_name}.customer_sv")
                .selectExpr("customer_id", "customer_unique_id", "customer_city", "customer_state")
        )

        stream_writer = (
            df_delta.writeStream
                .foreachBatch(data_upserter.upsert)
                .outputMode("update")
                .option("checkpointLocation", f"{self.checkpoint_base}/dim_customer_gd")
                .queryName("customers_upsert_stream")
        )

        spark.sparkContext.setLocalProperty("spark.scheduler.pool", "gold_p1")

        if once:
            return stream_writer.trigger(availableNow=True).start()
        else:
            return stream_writer.trigger(processingTime=processing_time).start()
        
    def upsert_dim_product_gd(self, once=True, processing_time="15 seconds"):
        from pyspark.sql import functions as F

        query = f"""
            MERGE INTO {self.catalog}.{self.db_gd_name}.dim_product_gd target
            USING product_delta source
            ON target.ProductID = source.product_id

            WHEN MATCHED THEN UPDATE SET
                target.CategoryName = source.category_name

            WHEN NOT MATCHED THEN INSERT (ProductID, CategoryName)
            VALUES (source.product_id, source.category_name)
        """

        data_upserter = Upserter(query, "product_delta")

        df_delta = (
            spark.readStream
                .table(f"{self.catalog}.{self.db_sv_name}.products_sv")
                .selectExpr("product_id", "product_category_name as category_name")
        )

        stream_writer = (
            df_delta.writeStream
                .foreachBatch(data_upserter.upsert)
                .outputMode("update")
                .option("checkpointLocation", f"{self.checkpoint_base}/dim_product_gd")
                .queryName("dim_product_gd_upsert_stream")
        )

        spark.sparkContext.setLocalProperty("spark.scheduler.pool", "gold_p2")

        if once:
            return stream_writer.trigger(availableNow=True).start()
        else:
            return stream_writer.trigger(processingTime=processing_time).start()

    def upsert_dim_seller_gd(self, once=True, processing_time="15 seconds"):
        from pyspark.sql import functions as F

        query = f"""
            MERGE INTO {self.catalog}.{self.db_gd_name}.dim_seller_gd target
            USING seller_delta source
            ON target.SellerID = source.seller_id

            WHEN MATCHED THEN UPDATE SET
                target.SellerCity = source.seller_city,
                target.SellerState = source.seller_state

            WHEN NOT MATCHED THEN INSERT (SellerID, SellerCity, SellerState)
            VALUES (source.seller_id, source.seller_city, source.seller_state)
        """

        data_upserter = Upserter(query, "seller_delta")

        df_delta = (
            spark.readStream
                .table(f"{self.catalog}.{self.db_sv_name}.sellers_sv")
                .selectExpr("seller_id", "seller_city", "seller_state")
        )

        stream_writer = (
            df_delta.writeStream
                .foreachBatch(data_upserter.upsert)
                .outputMode("update")
                .option("checkpointLocation", f"{self.checkpoint_base}/dim_seller_gd")
                .queryName("dim_seller_gd_upsert_stream")
        )

        spark.sparkContext.setLocalProperty("spark.scheduler.pool", "gold_p3")

        if once:
            return stream_writer.trigger(availableNow=True).start()
        else:
            return stream_writer.trigger(processingTime=processing_time).start()

    def upsert_dim_time_gd(self, once=True, processing_time="15 seconds"):
        from pyspark.sql import functions as F

        query = f"""
            MERGE INTO {self.catalog}.{self.db_gd_name}.dim_time_gd target
            USING time_delta source
            ON target.DateKey = source.DateKey

            WHEN MATCHED THEN UPDATE SET
                target.Date = source.Date,
                target.DayOfWeek = source.DayOfWeek,
                target.DayName = source.DayName,
                target.DayOfMonth = source.DayOfMonth,
                target.DayOfYear = source.DayOfYear,
                target.MonthName = source.MonthName,
                target.MonthOfYear = source.MonthOfYear,
                target.Quarter = source.Quarter,
                target.QuarterName = source.QuarterName,
                target.Year = source.Year,
                target.IsWeekday = source.IsWeekday

            WHEN NOT MATCHED THEN INSERT (
                DateKey, Date, DayOfWeek, DayName, DayOfMonth, DayOfYear,
                MonthName, MonthOfYear, Quarter, QuarterName, Year, IsWeekday
            )
            VALUES (
                source.DateKey, source.Date, source.DayOfWeek, source.DayName, source.DayOfMonth,
                source.DayOfYear, source.MonthName, source.MonthOfYear,
                source.Quarter, source.QuarterName, source.Year, source.IsWeekday
            )
        """

        data_upserter = Upserter(query, "time_delta")

        df_delta = (
            spark.readStream
                .table(f"{self.catalog}.{self.db_sv_name}.date_lookup_sv")
        )

        stream_writer = (
            df_delta.writeStream
                .foreachBatch(data_upserter.upsert)
                .outputMode("update")
                .option("checkpointLocation", f"{self.checkpoint_base}/dim_time_gd")
                .queryName("dim_time_gd_upsert_stream")
        )

        spark.sparkContext.setLocalProperty("spark.scheduler.pool", "gold_p4")

        if once:
            return stream_writer.trigger(availableNow=True).start()
        else:
            return stream_writer.trigger(processingTime=processing_time).start()

    def upsert_fact_sale_gd(self, once=True, processing_time="15 seconds"):
        from pyspark.sql import functions as F

        query = f"""
            MERGE INTO {self.catalog}.{self.db_gd_name}.fact_sale_gd AS target
            USING fact_sale_delta AS source
            ON target.order_item_id = source.order_item_id
            AND target.order_id = source.order_id
            AND target.product_id = source.product_id

            WHEN MATCHED THEN UPDATE SET
                target.PurchaseDateKey = source.PurchaseDateKey,
                target.DeliveredDateKey = source.DeliveredDateKey,
                target.EstimateDeliveredDateKey = source.EstimateDeliveredDateKey,
                target.CustomerKey = source.CustomerKey,
                target.ProductKey = source.ProductKey,
                target.SellerKey = source.SellerKey,
                target.Price = source.Price,
                target.FreightValue = source.FreightValue,
                target.TotalValue = source.TotalValue

            WHEN NOT MATCHED THEN INSERT (
                PurchaseDateKey, DeliveredDateKey, EstimateDeliveredDateKey,
                CustomerKey, ProductKey, SellerKey,
                order_item_id, order_id, product_id,
                Price, FreightValue, TotalValue
            )
            VALUES (
                source.PurchaseDateKey, source.DeliveredDateKey, source.EstimateDeliveredDateKey,
                source.CustomerKey, source.ProductKey, source.SellerKey,
                source.order_item_id, source.order_id, source.product_id,
                source.Price, source.FreightValue, source.TotalValue
            )
        """

        data_upserter = Upserter(query, "fact_sale_delta")

        sale_sv = spark.readStream.table(f"{self.catalog}.{self.db_sv_name}.sale_sv").alias("sale_sv")
        dim_customer = spark.table(f"{self.catalog}.{self.db_gd_name}.dim_customer_gd").alias("dim_customer")
        dim_seller = spark.table(f"{self.catalog}.{self.db_gd_name}.dim_seller_gd").alias("dim_seller")
        dim_product = spark.table(f"{self.catalog}.{self.db_gd_name}.dim_product_gd").alias("dim_product")

        df_delta = (
            sale_sv
            .join(dim_customer, F.col("sale_sv.customer_id") == F.col("dim_customer.CustomerID"))
            .join(dim_seller, F.col("sale_sv.seller_id") == F.col("dim_seller.SellerID"))
            .join(dim_product, F.col("sale_sv.product_id") == F.col("dim_product.ProductID"))
            .selectExpr(
                "cast(date_format(order_purchase_timestamp, 'yyyyMMdd') as int) as PurchaseDateKey",
                "case when order_delivered_customer_date is null then -1 else cast(date_format(order_delivered_customer_date, 'yyyyMMdd') as int) end as DeliveredDateKey",
                "cast(date_format(order_estimated_delivery_date, 'yyyyMMdd') as int) as EstimateDeliveredDateKey",

                "dim_customer.CustomerKey as CustomerKey",
                "dim_product.ProductKey as ProductKey",
                "dim_seller.SellerKey as SellerKey",

                "sale_sv.order_item_id",
                "sale_sv.order_id",
                "sale_sv.product_id",

                "sale_sv.price as Price",
                "sale_sv.freight_value as FreightValue",
                "(sale_sv.price + sale_sv.freight_value) as TotalValue"
            )
        )

        stream_writer = (
            df_delta.writeStream
                .foreachBatch(data_upserter.upsert)
                .outputMode("update")
                .option("checkpointLocation", f"{self.checkpoint_base}/fact_sale_gd")
                .queryName("fact_sale_gd_upsert_stream")
        )

        spark.sparkContext.setLocalProperty("spark.scheduler.pool", "gold_p3")

        if once:
            return stream_writer.trigger(availableNow=True).start()
        else:
            return stream_writer.trigger(processingTime=processing_time).start()
        
    def upsert_fact_order_gd(self, once=True, processing_time="15 seconds"):
        from pyspark.sql import functions as F

        query = f"""
            MERGE INTO {self.catalog}.{self.db_gd_name}.fact_order_gd AS target
            USING fact_order_delta AS source
            ON target.order_id = source.order_id

            WHEN MATCHED THEN UPDATE SET
                target.PurchaseDateKey      = source.PurchaseDateKey,
                target.DeliveredDateKey     = source.DeliveredDateKey,
                target.EstimateDeliveredDateKey = source.EstimateDeliveredDateKey,
                target.CustomerKey          = source.CustomerKey,
                target.ShipAmount           = source.ShipAmount,
                target.TotalProductValue    = source.TotalProductValue,
                target.TotalAmount          = source.TotalAmount,
                target.DeliveryActualDays   = source.DeliveryActualDays,
                target.DeliveryEstimateDays = source.DeliveryEstimateDays,
                target.ApproveDays          = source.ApproveDays

            WHEN NOT MATCHED THEN INSERT (
                PurchaseDateKey, DeliveredDateKey, EstimateDeliveredDateKey,
                CustomerKey, order_id,
                ShipAmount, TotalProductValue, TotalAmount,
                DeliveryActualDays, DeliveryEstimateDays, ApproveDays
            ) VALUES (
                source.PurchaseDateKey, source.DeliveredDateKey, source.EstimateDeliveredDateKey,
                source.CustomerKey, source.order_id,
                source.ShipAmount, source.TotalProductValue, source.TotalAmount,
                source.DeliveryActualDays, source.DeliveryEstimateDays, source.ApproveDays
            )
        """

        data_upserter = Upserter(query, "fact_order_delta")

        order_detail = spark.readStream.table(f"{self.catalog}.{self.db_sv_name}.order_detail_sv").alias("o")
        dim_customer = spark.table(f"{self.catalog}.{self.db_gd_name}.dim_customer_gd").alias("c")

        df_delta = (
            order_detail
            .join(dim_customer, F.col("o.customer_id") == F.col("c.CustomerID"))
            .groupBy(
                "o.order_purchase_timestamp",
                "o.order_delivered_customer_date",
                "o.order_estimated_delivery_date",
                "c.CustomerKey",
                "o.order_id",
                "o.order_approved_at"
            )
            .agg(
                F.sum("freight_value").alias("ShipAmount"),
                F.sum("price").alias("TotalProductValue")
            )
            .withColumn("TotalAmount", F.col("ShipAmount") + F.col("TotalProductValue"))
            .withColumn("PurchaseDateKey",
                (F.dayofmonth("order_purchase_timestamp")
                + F.month("order_purchase_timestamp") * 100
                + F.year("order_purchase_timestamp") * 10000)
            )
            .withColumn("DeliveredDateKey",
                F.when(F.col("order_delivered_customer_date").isNull(), F.lit(-1))
                .otherwise(
                    F.dayofmonth("order_delivered_customer_date")
                    + F.month("order_delivered_customer_date") * 100
                    + F.year("order_delivered_customer_date") * 10000
                )
            )
            .withColumn("EstimateDeliveredDateKey",
                F.dayofmonth("order_estimated_delivery_date")
                + F.month("order_estimated_delivery_date") * 100
                + F.year("order_estimated_delivery_date") * 10000
            )
            .withColumn("DeliveryActualDays",
                F.datediff("order_delivered_customer_date", "order_approved_at")
            )
            .withColumn("DeliveryEstimateDays",
                F.datediff("order_estimated_delivery_date", "order_approved_at")
            )
            .withColumn("ApproveDays",
                F.datediff("order_approved_at", "order_purchase_timestamp")
            )
            .selectExpr(
                "PurchaseDateKey",
                "DeliveredDateKey",
                "EstimateDeliveredDateKey",
                "CustomerKey",
                "order_id",
                "ShipAmount",
                "TotalProductValue",
                "TotalAmount",
                "DeliveryActualDays",
                "DeliveryEstimateDays",
                "ApproveDays"
            )
        )

        stream_writer = (
            df_delta.writeStream
                .foreachBatch(data_upserter.upsert)
                .outputMode("update")
                .option("checkpointLocation", f"{self.checkpoint_base}/fact_order_gd")
                .queryName("fact_order_gd_upsert_stream")
        )

        spark.sparkContext.setLocalProperty("spark.scheduler.pool", "gold_p4")

        if once:
            return stream_writer.trigger(availableNow=True).start()
        else:
            return stream_writer.trigger(processingTime=processing_time).start()
        
    def load_dim(self, once=True, processing_time="15 seconds"):
        print("Tiến hành load dữ liệu vào Gold Layer...")  
        self.upsert_dim_customer_gd(once, processing_time)
        self.upsert_dim_product_gd(once, processing_time)
        self.upsert_dim_seller_gd(once, processing_time)
        self.upsert_dim_time_gd(once, processing_time)
        if once:
            for stream in spark.streams.active:
                stream.awaitTermination()

    def load_fact(self, once=True, processing_time="15 seconds"):
        print("Tiến hành load dữ liệu vào Gold Layer...")  
        self.upsert_fact_order_gd(once, processing_time)
        self.upsert_fact_sale_gd(once, processing_time)
        if once:
            for stream in spark.streams.active:
                stream.awaitTermination()

In [0]:
gd = Gold("dev")
gd.cleanup()
gd.load_dim(once=True)

In [0]:
gd.load_fact(once=True)